In [31]:
import pandas as pd
import numpy as np
import re
import sys
from opencage.geocoder import OpenCageGeocode
from pprint import pprint

# np.set_printoptions(threshold=sys.maxsize)

In [29]:
inputFile = 'data_formatted.csv'
outputFile = 'data_formatted_2.csv'

In [30]:
# read
df = pd.read_csv(inputFile, sep=',')

In [ ]:
#shortens name: Lastname, F.S.
def shortenNames(columnName):
    column = df[columnName] 
    newValues = []
    wrong = []

    for x, string in column.iteritems():
        #checkes whether value is nan
        if (not string) or string != string:
            newValues.append(np.nan)
            continue
        end = []
            
        names = [name.strip() for name in string.split(';')]
        for i, name in enumerate(names):
            ns = name.split(', ')
            #ns has to have two parts: lastname, firstname
            if len(ns) == 1:
                wrong.append([x, i])
                end.append(name)
                continue
            firstname = ns[1].strip()
            lastname = ns[0].replace(',', '').strip()
           
            #checkes whether the value alreaddy has short parts
            if (re.search('.\.', firstname)):
                newFN = firstname.replace('-', '').strip()
                firstnames = newFN.split(' ')
                    
                fn = ''
                for piece in firstnames:
                    if piece == '':
                        continue
                    if (re.search('.\.', piece)):
                        fn += piece
                    else:
                        #first letter of name
                        fn += piece[0] + '.'
                end.append(lastname + ', ' + fn)
            else:
                newFN = firstname.replace('-', ' ') #.replace(' ', '')
                firstnames = newFN.split(' ')
                fn = ''
                for piece in firstnames:
                    if piece == '':
                        continue
                    if len(piece) == 1:
                        fn += piece + '.'
                    else:
                        #first letter of name
                        fn += piece[0] + '.'
                end.append(lastname + ', ' + fn)
        newValues.append(';'.join(end))
    return pd.Series(newValues), wrong

In [ ]:
%%time
authors, wrong = shortenNames('authors')
df['authors'] = authors

In [86]:
def cleanUpPlaces():
    places = df['place']
    newValues = []
    types = {}
    key = '0d9b85af587c4cbd8cf792e4a59c6497'
    geocoder = OpenCageGeocode(key)
    
    for x, string in places.iteritems():
        #checkes whether value is nan
        if (not string) or string != string:
            newValues.append(np.nan)
            continue
        
        place = places[x].replace('.', '').split(', ')
        newPlace = []
        for i, p in enumerate(place):
            query = p
            if query not in types:
                results = geocoder.geocode(query, pretty = '1', no_annotation = '1')
                if results == []:
                    types[query] = ''
                else:
                    types[query] = results[0]['components']['_type']
                    
            if (types[query] == '') and (len(place) == 1):
                newValues.append(np.nan)
            if (types[query] == 'city'): #or... state_district
                newPlace.append(query)
                
        newValues.append(', '.join(newPlace))
    return pd.Series(newValues)

In [ ]:
places = cleanUpPlaces()
df['place'] = places

In [85]:
cleanUpPlaces()

463

In [ ]:
shortenNames('editors')

In [ ]:
def split(columnName, seperator, newName = None, dropNaN=True):
    if newName == None:
        newName = columnName
    newFrame = []
    for _, row in df.iterrows():
        _id = row['id']
        item = str(row[columnName])
        if dropNaN and item == "nan":
            continue
        pieces = item.split(seperator)
        for piece in pieces:
            newFrame.append([_id, piece.strip()])
    return pd.DataFrame(newFrame, columns=['id', newName])

In [ ]:
illegalCharsRegex = '[^üéöíøóäñÜÖßáÄ¢Šç’´‐\w\'\-\s.,]'
def findIllegalNames(data, column, regex):
    copy = data[column].dropna()
    return copy[copy.str.contains(regex, regex=True, flags= re.I)]

In [ ]:
authors = split('authors', ';', 'name')
findIllegalNames(authors, 'name', illegalCharsRegex)

In [ ]:
editors = split('editors', ';', 'name')
findIllegalNames(editors, 'name', illegalCharsRegex)

In [ ]:
oldDepartmentNames = split('oldDepartmentNames', ',', 'name')
oldDepartmentNames['name'].unique()

In [ ]:
df['type'].unique()

In [ ]:
df['publisher'].dropna().unique()

In [ ]:
df['journal'].dropna().unique()

In [ ]:
findIllegalNames(df, 'startpage', '[^0-9]').unique()

In [ ]:
findIllegalNames(df, 'endpage', '[^0-9]').unique()

In [ ]:
df['issue'].dropna().unique()
# findIllegalNames(df, 'issue', '[^0-9]').unique()

In [ ]:
df['vol'].dropna().unique()

In [ ]:
df['year'].dropna().unique()

In [ ]:
findIllegalNames(df, 'place', illegalCharsRegex)

In [ ]:
df['link'].dropna().unique()

# findIllegalNames(df, 'issue', '^$http.*').unique()

In [ ]:
df['comment'].dropna().unique()

In [ ]:
df['keywordsAndPeerReview'].dropna().unique()

In [ ]:
df['DOI'].dropna().unique()

In [ ]:
df.loc[df['journal'].dropna().index]['type'].unique()

In [ ]:
# booktitle appears not only in 'inbook' but also in other types of publication
df.loc[df['booktitle'].dropna().index]['type'].unique()

In [ ]:
# df[~df['relation (= Serie)'].isnull()]
df['Serie'].dropna().unique().shape

In [ ]:
df[~df['Serie'].isnull()]['type'].unique()

In [ ]:
df['conference'].dropna().unique().shape

In [ ]:
df.loc[df['conference'].dropna().index]['type'].unique()

In [ ]:
df.loc[df['DOI'].dropna().index]['type'].unique()

In [ ]:
df.loc[df['startpage'].dropna().index]['type'].unique()

In [ ]:
df.to_csv(outputFile, encoding="utf8")